In [1]:
import pandas as pd
import json

In [1]:
from api_doi_collector import load_json, transform_json, concat_dfs

In [4]:
paths = [
    './openaire/openaire_dois.json',
    './openalex/openalex_dois.json',
    './zenodo/zenodo_dois.json'      
]
names = ['openaire', 'openalex', 'zenodo']

dfs = []
for path, name in zip(paths, names):
    data = load_json(path)
    df = transform_json(data, name)
    dfs.append(df)

df = concat_dfs(dfs)


In [5]:
df.shape

(51782, 2)

In [6]:
df = df.drop_duplicates('doi',keep='first')

In [7]:
df.shape

(45538, 2)

In [8]:
df['download_link'] = df['doi'].str.replace('https://doi.org/','https://sci-hub.hkvisa.net/1')

C:\Users\adriy\AppData\Local\Temp\ipykernel_4568\1261930972.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['download_link'] = df['doi'].str.replace('https://doi.org/','https://sci-hub.hkvisa.net/1')


In [9]:
df

,doi,name,download_link
0,https://doi.org/10.6084/m9.figshare.c.3804628_d2,openaire,https://sci-hub.hkvisa.net/110.6084/m9.figshar...
1,https://doi.org/10.6084/m9.figshare.c.3804628_...,openaire,https://sci-hub.hkvisa.net/110.6084/m9.figshar...
2,https://doi.org/10.6084/m9.figshare.c.3804628_...,openaire,https://sci-hub.hkvisa.net/110.6084/m9.figshar...
3,https://doi.org/10.6084/m9.figshare.c.3804628_d1,openaire,https://sci-hub.hkvisa.net/110.6084/m9.figshar...
4,https://doi.org/10.3389/fpls.2018.01030,openaire,https://sci-hub.hkvisa.net/110.3389/fpls.2018....
...,...,...,...
328,https://doi.org/10.5281/zenodo.3557376,zenodo,https://sci-hub.hkvisa.net/110.5281/zenodo.355...
329,https://doi.org/10.35940/ijeat.C5328.029320,zenodo,https://sci-hub.hkvisa.net/110.35940/ijeat.C53...
330,https://doi.org/10.21307/ijssis-2017-446,zenodo,https://sci-hub.hkvisa.net/110.21307/ijssis-20...
331,https://doi.org/10.5281/zenodo.1433824,zenodo,https://sci-hub.hkvisa.net/110.5281/zenodo.143...
